In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yaml

In [ ]:
def get_config(path: str) -> dict:
    """Open a yaml file and return the contents."""
    with open(path) as input_file:
        return yaml.safe_load(input_file)

The file paths and user inputs for the analysis of the Diamond 2017, Diamond 2021 and DESY 2021 experiments are included in the `yaml` configuration files, to record the inputs of the analysis.

The information in the `yaml` file can be accessed like this.

In [ ]:
# config_path = "yaml/config_desy_2020_results.yaml"
# config_path = "yaml/config_desy_2021_results.yaml"
config_path = "yaml/config_diamond_2022_results_slow_acquisition.yaml"

config = get_config(config_path)

Input the order number for the test you would like to look at. *Note, the start of a list in python is zero.* 

The order number will set the experiment number, defining the input and output file name, as well as defining the associated inputs from the yaml file, which will be used to analyse and plot the data for the test.

In [ ]:
order_number = 2
#between 0 and 11 for DESY 2020 experiment, 0 and 3 for DESY 2021 experiment, 0 and 23 for Diamond 2022 experiment

experiment_name = config["results_setup"]["experiment_name"][order_number]
print("The experiment name being analysed is: ", experiment_name)

experiment_number = config["results_setup"]["experiment_number"][order_number]
print("The experiment number being analysed is: ", experiment_number)

Load the data.

In [ ]:
input_filepath = config["results_setup"]["input_filepath"].format(experiment_name = experiment_name, 
                                                                  experiment_number = experiment_number)
output_filepath = config["results_setup"]["output_filepath"].format(experiment_name = experiment_name, 
                                                                    experiment_number = experiment_number)

data_array = np.loadtxt(input_filepath, usecols=(2,4,6,8,10,12))
file_length = len(data_array)

data_Rwp = np.loadtxt(input_filepath, usecols=(2))
data_GOF = np.loadtxt(input_filepath, usecols=(4))
data_alpha_a = np.loadtxt(input_filepath, usecols=(6))
data_alpha_c = np.loadtxt(input_filepath, usecols=(8))
data_alpha_wt = np.loadtxt(input_filepath, usecols=(10))
data_beta_a = np.loadtxt(input_filepath, usecols=(12))
data_beta_wt = np.loadtxt(input_filepath, usecols=(14))

frame_number = np.arange(1,file_length+1,1)

print("The input filepath is: ", input_filepath, sep='\n', end='\n\n')
print("And it contains", file_length, " analysed spectra from ", file_length, " images.")

Are you applying limits to the data?

In [ ]:
limits = config["results_setup"]["plot_bounds"]["limits"]

if limits:
    deformation_start = config["results_setup"]["plot_bounds"]["deformation_start"][order_number]
    deformation_end = config["results_setup"]["plot_bounds"]["deformation_end"][order_number]
    rwp_min = config["results_setup"]["plot_bounds"]["rwp_min"][order_number]
    rwp_max = config["results_setup"]["plot_bounds"]["rwp_max"][order_number]
    gof_min = config["results_setup"]["plot_bounds"]["gof_min"][order_number]
    gof_max = config["results_setup"]["plot_bounds"]["gof_max"][order_number]
    alpha_c_min = config["results_setup"]["plot_bounds"]["alpha_c_min"][order_number]
    alpha_c_max = config["results_setup"]["plot_bounds"]["alpha_c_max"][order_number]
    alpha_a_min = config["results_setup"]["plot_bounds"]["alpha_a_min"][order_number]
    alpha_a_max = config["results_setup"]["plot_bounds"]["alpha_a_max"][order_number]  
    beta_a_min = config["results_setup"]["plot_bounds"]["beta_a_min"][order_number]
    beta_a_max = config["results_setup"]["plot_bounds"]["beta_a_max"][order_number]
    
    print("The following limits are being applied to the subsequent plots...", end='\n\n')
    print("The deformation start is frame number: ", deformation_start, " and the deformation end is : ", deformation_end, end='\n\n')
    print("The Rwp error minimum is: ", rwp_min, " and the Rwp maximum is: ", rwp_max, end='\n\n')
    print("The Goodness of Fit (GOF) minimum is: ", gof_min, " and the GOF maximum is: ", gof_max, end='\n\n')
    print("The alpha_c lattice parameter minimum is: ", alpha_c_min, " and the alpha_c maximum is: ", alpha_c_max, end='\n\n')
    print("The alpha_a lattice parameter minimum is: ", alpha_a_min, " and the alpha_a maximum is: ", alpha_a_max, end='\n\n')
    print("The beta_a lattice parameter minimum is: ", beta_a_min, " and the beta_a maximum is: ", beta_a_max)

elif not limits:
    print("Limits are NOT being applied to the subsequent plots.")

Plot commands.

In [ ]:
plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)
plt.rc('legend',fontsize=20)
plt.rc('axes',linewidth=2)
plt.rc('xtick.major',width=2,size=10)
plt.rc('xtick.minor', width=2, size=5)
plt.rc('ytick.major',width=2,size=10)
plt.rc('ytick.minor',width=2,size=5)

Plot the $\mathrm{R_{wp}}$ value

In [ ]:
plt.figure(figsize=(10,8))
plt.minorticks_on()

plt.plot(frame_number[:],data_Rwp[:],color='black',linewidth=5);

plt.xlabel('Frame Number', fontsize=28)
plt.ylabel('$\mathrm{R_{wp}}$ Value', fontsize=28)

if limits:
    plt.ylim(rwp_min,rwp_max)

elif not limits:
    plt.ylim(0)
    
plt.xlim(0,file_length)

plt.savefig("{output_filepath}/Rwp_{experiment_number}.png".format(output_filepath = output_filepath, 
                                                                   experiment_number = experiment_number))
plt.show()

Plot the goodness of fit.

In [ ]:
plt.figure(figsize=(10,8))
plt.minorticks_on()

plt.plot(frame_number[:],data_GOF[:],color='darkviolet',linewidth=5 );

plt.xlabel('Frame Number', fontsize=28)
plt.ylabel('GOF Value', fontsize=28)

if limits:
    plt.ylim(gof_min,gof_max)

elif not limits:
    plt.ylim(0)
    
plt.xlim(0,file_length)

plt.savefig('{output_filepath}/GOF_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                   experiment_number = experiment_number))
plt.show()

Plot the $\alpha$ and $\beta$ lattice parameters for the entire experiment.

In [ ]:
frame_acquisition = config["results_setup"]["frame_acquisition"]

if frame_acquisition:

    seconds = frame_number / frame_acquisition

    fig,((ax1),(ax2),(ax3)) = plt.subplots(3, 1, figsize=(20,10))

    ax1.plot(seconds[:],data_alpha_c[:],label='${\\alpha_c}$',color='red',linewidth=5)    
    ax1.minorticks_on()
    ax1.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
#     ax1.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

    ax2.set_ylabel('Lattice Parameter, ${\\AA}$ ', fontsize=36)
    ax2.yaxis.set_label_coords(-0.07,0.5)
    ax2.plot(seconds[:],data_alpha_a[:], label='${\\alpha_a}$', color='green',linewidth=5)
    ax2.minorticks_on()
    ax2.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
#     ax2.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

    ax3.plot(seconds[:],data_beta_a[:],label='${\\beta_a}$',color='blue',linewidth=5)
    ax3.set_xlabel('Time, s', fontsize=36)
    ax3.minorticks_on()
    ax3.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
#     ax3.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

    plt.savefig('{output_filepath}/lattice_full_seconds_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                 experiment_number = experiment_number))
    plt.show()
    
elif not frame_acquisition:
    fig,((ax1),(ax2),(ax3))=plt.subplots(3, 1, figsize=(20,10))

    ax1.plot(frame_number[:],data_alpha_c[:],label='${\\alpha_c}$',color='red',linewidth=5)
    ax1.minorticks_on()
    ax1.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
#     ax1.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

    ax2.set_ylabel('Lattice Parameter, ${\\AA}$ ', fontsize=36)
    ax2.yaxis.set_label_coords(-0.07,0.5)
    ax2.plot(frame_number[:],data_alpha_a[:], label='${\\alpha_a}$', color='green',linewidth=5)
    ax2.minorticks_on()
    ax2.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
#     ax2.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

    ax3.plot(frame_number[:],data_beta_a[:],label='${\\beta_a}$',color='blue',linewidth=5)
    ax3.set_xlabel('Frame Number', fontsize=36)
    ax3.minorticks_on()
    ax3.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
#     ax3.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

    plt.savefig('{output_filepath}/lattice_full_frames_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                experiment_number = experiment_number))
    plt.show()

Plot the $\alpha$ and $\beta$ lattice parameters during loading.

In [ ]:
limit_warning = "Limits have not been applied in the yaml file, please create limits for start and end of deformation, and limits for the lattice cell."

if frame_acquisition:
    
    if limits: 
        
        fig,((ax1),(ax2),(ax3)) = plt.subplots(3, 1, figsize=(20,10))

        ax1.plot(seconds[:],data_alpha_c[:],label='${\\alpha_c}$',color='red',linewidth=5)
        ax1.set_xlim(deformation_start/frame_acquisition, deformation_end/frame_acquisition)
        ax1.set_ylim(alpha_c_min, alpha_c_max)
        ax1.minorticks_on()
#         ax1.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
        ax1.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

        ax2.set_ylabel('Lattice Parameter, ${\\AA}$ ', fontsize=36)
        ax2.yaxis.set_label_coords(-0.07,0.5)
        ax2.plot(seconds[:],data_alpha_a[:], label='${\\alpha_a}$', color='green',linewidth=5)
        ax2.set_xlim(deformation_start/frame_acquisition, deformation_end/frame_acquisition)
        ax2.set_ylim(alpha_a_min, alpha_a_max)
        ax2.minorticks_on()
#         ax2.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
        ax2.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

        ax3.plot(seconds[:],data_beta_a[:],label='${\\beta_a}$',color='blue',linewidth=5)
        ax3.set_xlabel('Time, s', fontsize=36)
        ax3.set_xlim(deformation_start/frame_acquisition, deformation_end/frame_acquisition)
        ax3.set_ylim(beta_a_min, beta_a_max)
        ax3.minorticks_on()
#         ax3.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
        ax3.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

        plt.savefig('{output_filepath}/lattice_deform_seconds_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                       experiment_number = experiment_number))
        plt.show()
        
    elif not limits:
        
        print(limit_warning)
        
elif not frame_acquisition:
    
    if limits: 
        
        fig,((ax1),(ax2),(ax3)) = plt.subplots(3, 1, figsize=(20,10))

        ax1.plot(frame_number[:],data_alpha_c[:],label='${\\alpha_c}$',color='red',linewidth=5)
        ax1.set_xlim(deformation_start, deformation_end)
        ax1.set_ylim(alpha_c_min, alpha_c_max)
        ax1.minorticks_on()
#         ax1.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
        ax1.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

        ax2.set_ylabel('Lattice Parameter, ${\\AA}$ ', fontsize=36)
        ax2.yaxis.set_label_coords(-0.07,0.5)
        ax2.plot(frame_number[:],data_alpha_a[:], label='${\\alpha_a}$', color='green',linewidth=5)
        ax2.set_xlim(deformation_start, deformation_end)
        ax2.set_ylim(alpha_a_min, alpha_a_max)
        ax2.minorticks_on()
#         ax2.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
        ax2.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

        ax3.plot(frame_number[:],data_beta_a[:],label='${\\beta_a}$',color='blue',linewidth=5)
        ax3.set_xlabel('Frame Number', fontsize=36)
        ax3.set_xlim(deformation_start, deformation_end)
        ax3.set_ylim(beta_a_min, beta_a_max)
        ax3.minorticks_on()
#         ax3.legend(loc=1, bbox_to_anchor=(0.125,1), prop={'size': 25}) #top left
        ax3.legend(loc=1, bbox_to_anchor=(0.125,0.42), prop={'size': 25}) #bottom left

        plt.savefig('{output_filepath}/lattice_deform_frames_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                      experiment_number = experiment_number))
        plt.show()
        
    elif not limits:
        
        print(limit_warning)

Plot the $\alpha$ and $\beta$ phase fraction for the entire experiment.

In [ ]:
if frame_acquisition:
    
    second_length = file_length / frame_acquisition

    plt.figure(figsize = (10,8))

    plt.minorticks_on()
    plt.plot(seconds[:],data_alpha_wt[:],label='${\\alpha}$-phase', color='red',linewidth=8);
    plt.plot(seconds[:],data_beta_wt[:],label='${\\beta}$-phase',color='blue',linewidth=8)

    plt.xlabel('Time, s', fontsize=28)
    plt.ylabel('Phase Fraction, wt.%', fontsize=28)

    plt.xlim(0,second_length)
    plt.ylim(0,100)

    plt.legend()
    
    plt.savefig('{output_filepath}/phasefrac_full_seconds_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                   experiment_number = experiment_number))
    plt.show()
    
elif not frame_acquisition:
    
    plt.figure(figsize=(10,8))

    plt.minorticks_on()
    plt.plot(frame_number[:],data_alpha_wt[:], label='${\\alpha}$-phase', color='red',linewidth=8);
    plt.plot(frame_number[:],data_beta_wt[:],label='${\\beta}$-phase',color='blue',linewidth=8)

    plt.xlabel('Frame Number', fontsize=28)
    plt.ylabel('Phase Fraction, wt.%', fontsize=28)

    plt.xlim(0,file_length)
    plt.ylim(0,100)

    plt.legend()

    plt.savefig('{output_filepath}/phasefrac_full_frames_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                  experiment_number = experiment_number))
    plt.show()

Plot the $\alpha$ and $\beta$ phase fraction during loading.

In [ ]:
if frame_acquisition:
    
    if limits:
        
        second_length = file_length / frame_acquisition

        plt.figure(figsize = (10,8))

        plt.minorticks_on()
        plt.plot(seconds[:],data_alpha_wt[:],label='${\\alpha}$-phase', color='red',linewidth=8);
        plt.plot(seconds[:],data_beta_wt[:],label='${\\beta}$-phase',color='blue',linewidth=8)

        plt.xlabel('Time, s', fontsize=28)
        plt.ylabel('Phase Fraction, wt.%', fontsize=28)

        plt.xlim(deformation_start/frame_acquisition,deformation_end/frame_acquisition)
        plt.ylim(0,100)

        plt.legend()

        plt.savefig('{output_filepath}/phasefrac_deform_seconds_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                         experiment_number = experiment_number))
        plt.show()

    elif not limits:

        print(limit_warning)

elif not frame_acquisition:

    if limits:
        
        plt.figure(figsize=(10,8))

        plt.minorticks_on()
        plt.plot(frame_number[:],data_alpha_wt[:], label='${\\alpha}$-phase', color='red',linewidth=8);
        plt.plot(frame_number[:],data_beta_wt[:],label='${\\beta}$-phase',color='blue',linewidth=8)

        plt.xlabel('Frame Number', fontsize=28)
        plt.ylabel('Phase Fraction, wt.%', fontsize=28)

        plt.xlim(deformation_start,deformation_end)
        plt.ylim(0,100)

        plt.legend()

        plt.savefig('{output_filepath}/phasefrac_deform_frames_{experiment_number}.png'.format(output_filepath = output_filepath, 
                                                                                        experiment_number = experiment_number))
        plt.show()

    elif not limits:

        print(limit_warning)